# Set-up

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)


# Guy's data - first batch

In [3]:
guys = pd.read_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/raw/08_03_24 GUYS HOPITAL-SANGER PHENOTYPE.csv", header = 1)

guys = guys.drop("Patient Information", axis =1)

guys = guys.T

guys.columns = guys.iloc[0]

guys = guys.dropna(axis = 0, how = 'all')

guys = guys.drop(index = ["Patient_ID", "Unnamed: 2", "L1"])

guys = guys.reset_index(names = "Patient_ID")

guys["Site"] = "Guys"

guys["Recruit_centre"] = "Kings"

drop_patients = ["EQTL0166p", "EQTL0180p", "EQTL0123p", "EQTL0110p", "EQTL0105p"] 

# 0166p and 0180p duplicated in other spreadsheet, 
# 123p, 110p and 034p are multiple samples from same individ, keep 034p (because 
# sample collection date in metadata matches clinical info for this sample)
# 105p and 134p are dups, keep 134p

guys = guys[~guys["Patient_ID"].isin(drop_patients)]

# EQTL0156p also in guys_new, need to check because some values differ 
# (but guys_new appears to have more appropriate test dates)
guys = guys.query("Patient_ID != 'EQTL0156p'") 

# Guy's data - second batch

In [4]:
guys_new = pd.read_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/raw/14_03_24 GUYS HOPITAL-SANGER PHENOTYPE TEMPLATE UPDATED.csv", header = 1)

guys_new = guys_new.drop("Patient Information", axis =1)

guys_new = guys_new.T

guys_new.columns = guys_new.iloc[0]

guys_new = guys_new.dropna(axis = 0, how = 'all')

guys_new = guys_new.drop(index = ["Patient_ID", "Unnamed: 2", "L1"])

guys_new = guys_new.reset_index(names = "Patient_ID")

guys_new["Site"] = "Guys"

guys_new["Recruit_centre"] = "Kings"


# King's data

In [5]:
kings = pd.read_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/raw/21_02_2024 KINGS HOPITAL-SANGER PHENOTYPE.csv", header = 1)

kings = kings.drop("Patient Information", axis =1)

kings = kings.T

kings.columns = kings.iloc[0]

kings = kings.dropna(axis = 0, how = 'all')

kings = kings.drop(index = ["Patient_ID", "Unnamed: 2", "L1"])

kings = kings.reset_index(names = "Patient_ID")

kings["Site"] = "Kings"

kings["Recruit_centre"] = "Kings"

# Combine Guy's and King's

In [6]:
kings_both = pd.concat([guys, guys_new, kings])
kings_both.SLEDAI_2K = kings_both.SLEDAI_2K.astype("Int64")
kings_both = kings_both.rename({"C4 _level":"C4_level"}, axis = 1)
kings_both = kings_both.rename({"Year of Birth":"Year_of_Birth"}, axis = 1)


In [7]:
kings_both = kings_both.replace("Unspecified", np.nan).infer_objects(copy=False)


/tmp/ipykernel_607266/4056398044.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  kings_both = kings_both.replace("Unspecified", np.nan).infer_objects(copy=False)


In [8]:
kings_both = kings_both.rename({"Date of sampling":"Sample_date"}, axis = 1)
kings_both["Year_of_Birth"] = kings_both["Year_of_Birth"].astype(int)
kings_both["Sample_year"] = kings_both["Sample_date"].str.split("/").str[2]
kings_both["Sample_year"] = kings_both["Sample_year"].astype(int)
kings_both["Age_at_sampling"] = kings_both["Sample_year"] - kings_both["Year_of_Birth"]

kings_both["Diagnosis_year"] = kings_both["Diagnosis_date"]
kings_both["Diagnosis_year"] = kings_both["Diagnosis_year"].replace({"Sep-19":"2019", "Aug-21":"2021",
                                                                       "Jan-23":"2023"})
kings_both["Diagnosis_year"] = kings_both["Diagnosis_year"].astype(float)


# Output transposed King's

In [9]:
kings_both_t = kings_both.T 

In [10]:
kings_both_t.columns = kings_both_t.iloc[0]
kings_both_t = kings_both_t[1:]

In [11]:
kings_both_t.to_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/raw/Kings_Guys_combined.csv", na_rep='NA')

# Imperial data

In [12]:
# imperial = pd.read_excel("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/raw/OTAR_Phenotyping_ICL_300424.xlsx")

# imperial.to_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/raw/OTAR_Phenotyping_ICL_300424.csv", index = False)
#L116 and S79 are dups, keep S79

In [13]:
imperial = pd.read_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/raw/OTAR_Phenotyping_ICL_300424.csv")

In [14]:
imperial["Site"] = "Imperial"

imperial["Recruit_centre"] = "Imperial"

imperial = imperial.rename({"Patient_N":"Patient_ID"}, axis = 1)

imperial["Patient_ID"] = imperial["Patient_ID"].str.replace(" ", "")

In [15]:
imperial = imperial.replace({"Not Done":np.nan, "ND":np.nan, "NaT":np.nan, "NK":np.nan}).infer_objects(copy=False)


In [16]:
imperial["Sample_year"] = imperial["Sample_date"].str.split("-").str[0]
imperial["Sample_year"] = imperial["Sample_year"].astype(int)
imperial["Diagnosis_year"] = imperial["Diagnosis_date"].str.split("-").str[0]
imperial["Diagnosis_year"] = imperial["Diagnosis_year"].astype(float)
imperial["Year_of_Birth"] = imperial["Sample_year"] - imperial["Age_at_sampling"]

In [17]:
imperial

,Patient_ID,Sample_ID,Sample_date,Sex,Ethnicity_cerner,Age_at_sampling,Diagnosis_date,LN,LN_date,LN_Class,Kidney_biopsies,Lupus_status,MMF,MMF_dose,Pred,Pred_dose,HCQ,HCQ_dose,AZA,AZA_dose,MTX,MTX_dose,TAC,TAC_dose,ASP,ASP_dose,CYC_ever,CYC_dose,CYC_date,CYC_cycles,BEL_ever,BEL_dose,BEL_date,RTX_ever,RTX_dose,RTX_date,RTX_cycles,OFA_ever,OFA_dose,OFA_date,OFA_cycles,dsDNA_titre,dsDNA_units,dsDNA_date,dsDNA Ab_ever,ANA,ANA_ever,Ribosomal_P,Ro_52,Ro_60,La,Centromere,Sm,Sm_RNP,RNP_68,RNP_A,Scl70,Jo_1,MPO,PR3,LAC_status,ACA_status,ACA_IgG,ACA_IgM,B2G_status,B2G_IgG,B2G_IgM,Vitamin_D,Vitamin_D_date,TSH,TSH_date,C3_level,C4_level,Comp_date,uPCR,uPCR_units,uPCR_date,Albumin,Creatinine,eGFR,Haemoglobin,Lymphopenia_now,Lymphopenia_ever,Lymphocyte_count,Neutrophil_count,Monocyte_count,FBC_Biochem_date,B_cell_n,B_cell_%,CD4_n,CD4_%,CD8_n,CD8_%,Lymph_subsets_date,IFN_score,IFN_status,SLEDAI_2K,SLEDAI_2KG,SLEDAI_2K_Clinical,SLEDAI_2K_Comments,C3_status,C4_status,dsDNA_SLEDAI_status,ACR_Malar,ACR_Discoid,ACR_Photosensitivity,ACR_oral_ulcers,ACR_Arthritis,ACR_serositis,ACR_seizures/psychosis,ACR_Haem,ACR_Immunologic,Organ_CVS,Organ_Arterial,Organ_Venous,Organ_Lung_Fibrosis,Any_other_comorbidities,Thyroid_Disease_Hashimotos,Graves_Disease,Type_I_Diabetes,Rheumatoid_Arthritis,Multiple_Sclerosis,KBX_2,KBX_2_LN_Class,KBX_3,KBX_3_LN_Class,KBX_4,KBX_4_LN_Class,KBX_5,KBX_5_LN_Class,IgA,IgG,IgM,Ig_date,Site,Recruit_centre,Sample_year,Diagnosis_year,Year_of_Birth
0,L1,L1_040521,2021-05-04,F,Other.anyotherethnicgroup,50,2014-07-02 00:00:00,YES,2014-07-28 00:00:00,IV-S_(A/C),1,ACTIVE,YES,1g_BD,YES,5mg_OD,YES,200mg_BD,NO,NaN,NO,NaN,NO,NaN,NO,NaN,NO,NaN,NaN,0,NO,NaN,NaN,YES,1g,2020-09-25 00:00:00,2,NO,NaN,NaN,0,163,IUperML,sample_date,POSITIVE,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,37.9,2020-08-03 00:00:00,2.79,2021-05-06 00:00:00,1.35,0.24,sample_date,10,MGperMMOL,sample_date,41,60,>90,139,NO,YES,2.9,5.7,0.7,sample_date,29,1,1187,40.7,1088,40.7,sample_date,0.353,IFN-neg,8,10,6,Arthritis_Ulcers,NORMAL,NORMAL,HIGH,NO,NO,NO,YES,YES,YES,NO,YES,YES,NO,NO,NO,NO,TTP_2006,NO,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.05,10.9,0.36,sample_date,Imperial,Imperial,2021,2014.0,1971
1,L100,L100_020321,2021-03-02,F,White.Other,37,NaN,NO,NaN,NaN,0,INACTIVE,NO,NaN,YES,2mg_OD,YES,200mg_BD,NO,NaN,NO,NaN,NO,NaN,NO,NaN,NO,NaN,NaN,0,NO,NaN,NaN,NO,NaN,NaN,0,NO,NaN,NaN,0,7,IUperML,2021-03-16 00:00:00,POSITIVE,NaN,POSITIVE,NEGATIVE,POSITIVE,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,17.7,2022-05-12 00:00:00,0.46,2022-05-12 00:00:00,1.4,0.35,sample_date,7,MGperMMOL,sample_date,41,73,>90,117,NO,NO,2.0,2.1,0.2,sample_date,191,6.7,1123,40.5,1389,50.2,2020-06-02 00:00:00,2.837,IFN-high,0,0,0,NaN,NORMAL,NORMAL,NORMAL,NO,NO,NO,YES,NO,YES,NO,NO,YES,NO,NO,NO,NO,NaN,NO,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.83,17.2,0.33,2021-02-02 00:00:00,Imperial,Imperial,2021,NaN,1984
2,L101,L101_020321,2021-03-02,F,Black.Caribbean,19,2002-02-13 00:00:00,NO,NaN,NaN,0,INACTIVE,NO,NaN,YES,5mg_OD,YES,200mg_OD,NO,NaN,NO,NaN,NO,NaN,NO,NaN,NO,NaN,NaN,0,NO,NaN,NaN,NO,NaN,NaN,0,NO,NaN,NaN,0,9,IUperML,sample_date,NEGATIVE,POSITIVE,POSITIVE,NEGATIVE,POSITIVE,POSITIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,NEGATIVE,POSITIVE,POSITIVE,NEGATIVE,POSITIVE,POSITIVE,NEGATIVE,POSITIVE,74.2,2021-11-16 00:00:00,1.99,sample_date,1.07,0.28,sample_date,7,MGperMMOL,sample_date,45,60,>90,118,NO,YES,2.3,1.2,0.3,sample_date,125,6.6,1226,66.8,347,18.9,2021-11-16 00:00:00,2.488,IFN-high,0,2,0,NaN,NORMAL,NORMAL,NORMAL,NO,NO,NO,NO,YES,NO,NO,YES,YES,NO,NO,NO,NO,DB_anaemia_Vitiligo_Osteoporosis,NO,NO,NO,NO,NO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.88,15.7,1.98,sample_date,Imperial,Imperial,2021,2002.0,2002
3,L105,L10

# Combine Imperial and Guy's/King's

In [18]:
both_sites = pd.concat([kings_both, imperial], join = "outer").copy()

In [19]:
both_sites = both_sites.rename({" B_cell_% ":"B_cell_%"}, axis = 1)

both_sites.columns = both_sites.columns.str.replace(" ", "_")


In [20]:
both_sites.Recruit_centre.unique()

array(['Kings', 'Imperial'], dtype=object)

In [21]:
both_sites["Years_from_diagnosis"] = both_sites["Sample_year"] - both_sites["Diagnosis_year"]

In [22]:
both_sites["Age_at_diagnosis"] = both_sites["Diagnosis_year"] - both_sites["Year_of_Birth"] 

# Tidy up IDs to match metadata 
Some metadata sample IDs have a trailing "r" which the clinical data doesn't have

In [23]:
both_sites.Patient_ID = both_sites.Patient_ID.str.replace("p", "P")

patient_ids = both_sites["Patient_ID"]

tidy_ids = list()

for x in patient_ids:
    if len(x) == 9 and x[4] == '0':
        new_x = x[:4] + x[5:]
        tidy_ids.append(new_x)
    else:
        tidy_ids.append(x)
        
both_sites.insert(loc=1, column='externalID', value=tidy_ids)

# Make case and NA usage consistent 

In [24]:
both_sites = both_sites.reset_index(drop = True)
both_sites = both_sites.rename({"C4 _level":"C4_level"}, axis = 1)

with pd.option_context('future.no_silent_downcasting', True):
    both_sites = both_sites.replace({"Not Done":np.nan, "ND":np.nan, "NaT":np.nan, "NK":np.nan}).infer_objects(copy=False)
    both_sites = both_sites.replace({"POSITIVE":"Positive", "NEGATIVE":"Negative"}).infer_objects(copy=False)
    both_sites = both_sites.replace({"YES":"Yes", "NO":"No"}).infer_objects(copy=False)

# Sort dtypes

In [25]:
both_sites.C3_level = both_sites.C3_level.astype(float)
both_sites.C4_level = both_sites.C4_level.astype(float)
both_sites.uPCR = both_sites.uPCR.str.replace("<", "") # need to decide what to do with < values
both_sites.uPCR = both_sites.uPCR.astype(float)
both_sites.Albumin = both_sites.Albumin.astype(float)
both_sites.Creatinine = both_sites.Creatinine.astype(float)
both_sites.eGFR = both_sites.eGFR.replace(">90", 90)
both_sites.eGFR = both_sites.eGFR.astype(float)
both_sites.Lymphocyte_count = both_sites.Lymphocyte_count.astype(float)
both_sites.SLEDAI_2K = both_sites.SLEDAI_2K.astype("float")
both_sites.SLEDAI_2K_Clinical = both_sites.SLEDAI_2K_Clinical.astype("float")


# Condense lupus nephritis classes

In [26]:
both_sites["LN_Class"].unique()

array(['Class III', nan, 'Class IV', 'III/IV', 'IV/V', 'III/V', 'IV', 'V',
       'II ( and FSGS)', 'IV-S_(A/C)', 'IV-S+V', 'IV-S_(A/C)+V',
       'IV_(A/C)+V', 'III_(A/C)', 'IV-G_(A/C)', 'IV_(A/C)', 'III_(A)+V',
       'IV+V', 'III+V', 'IV-G_(A)', 'III_(C)', 'III', 'II', 'III+IV ',
       'III_(A/C)+V', 'IV-G', 'III_(A)', 'III-S_(A)+V', 'IV-S',
       'IV-S_(C)', 'MN_300813', 'IV ', 'MCGN', 'III+IV', 'IV-S_(A)',
       'IV+III', 'II+V', 'V_transplant'], dtype=object)

In [27]:
both_sites["LN_Class_tidy"] = both_sites["LN_Class"].str.replace("Class","")
both_sites["LN_Class_tidy"] = both_sites["LN_Class_tidy"].str.replace(" ","")
both_sites["LN_Class_tidy"] = both_sites["LN_Class_tidy"].replace({"III/V":"III+V", "IV/V":"IV+V", 
                                                         "III/IV":"III+IV", "IV+III":"III+IV"})

In [28]:
both_sites["LN_Class_tidy"].unique()

array(['III', nan, 'IV', 'III+IV', 'IV+V', 'III+V', 'V', 'II(andFSGS)',
       'IV-S_(A/C)', 'IV-S+V', 'IV-S_(A/C)+V', 'IV_(A/C)+V', 'III_(A/C)',
       'IV-G_(A/C)', 'IV_(A/C)', 'III_(A)+V', 'IV-G_(A)', 'III_(C)', 'II',
       'III_(A/C)+V', 'IV-G', 'III_(A)', 'III-S_(A)+V', 'IV-S',
       'IV-S_(C)', 'MN_300813', 'MCGN', 'IV-S_(A)', 'II+V',
       'V_transplant'], dtype=object)

# Adjust eGFR values

In [29]:
#account for the fact that Imperial does not calculate eGFR > 90
both_sites["eGFR_adjust"] = np.where((both_sites['eGFR'] > 90), 90, both_sites['eGFR'] )



# Calculate clinical SLEDAI

In [30]:
def calculate_clinical_sledai(df):
    new_sledai = df["SLEDAI_2K"]
    if df["C3_status"] == "LOW" or df["C4_status"] == "LOW":
        new_sledai = new_sledai - 2
    if df["dsDNA_SLEDAI_status"] == "HIGH":
        new_sledai = new_sledai - 2
    if new_sledai < 0 :
        return 0
    else:
        return new_sledai

    

In [31]:
both_sites["cSLEDAI_2K"] = both_sites.apply(calculate_clinical_sledai, axis = 1)


In [32]:
list(both_sites.columns)

['Patient_ID',
 'externalID',
 'Date_of_consent',
 'Sample_date',
 'Year_of_Birth',
 'Ethnicity_cerner',
 'Diagnosis_date',
 'LN',
 'LN_date',
 'LN_Class',
 'Kidney_biopsies',
 'Organ_CVS',
 'Organ_Arterial',
 'Organ_Venous',
 'Organ_Lung_Fibrosis',
 'Multiple_Sclerosis',
 'Thyroid_Disease_Hashimotos',
 'Graves_Disease',
 'Type_I_Diabetes',
 'Rheumatoid_Arthritis',
 'Any_other_comorbidities',
 'MMF',
 'MMF_dose',
 'Pred',
 'Pred_dose',
 'HCQ',
 'HCQ_dose',
 'AZA',
 'AZA_dose',
 'MTX',
 'MTX_dose',
 'TAC',
 'TAC_dose',
 'ASP',
 'ASP_dose',
 'CYC_ever',
 'CYC_dose',
 'CYC_date',
 'CYC_cycles',
 'BEL_ever',
 'BEL_dose',
 'BEL_date',
 'RTX_ever',
 'RTX_dose',
 'RTX_date',
 'RTX_cycles',
 'OFA_ever',
 'OFA_dose',
 'OFA_date',
 'OFA_cycles',
 'dsDNA',
 'dsDNA_titre',
 'dsDNA_units',
 'dsDNA_Ab_ever',
 'dsDNA_date',
 'ANA',
 'ANA_ever',
 'Ribosomal_P',
 'Ro_52',
 'Ro_60',
 'La',
 'Centromere',
 'Sm',
 'Sm_RNP',
 'RNP_68',
 'RNP_A',
 'Scl70',
 'Jo_1',
 'MPO',
 'PR3',
 'LAC_status',
 'ACA_statu

# Re-order columns

In [33]:
new_order = ['Patient_ID',
 'externalID',
 'Sample_ID',
 'Site',
 'Recruit_centre',
 'Date_of_consent',
 'Sample_date',
 'Sample_year',
 'Year_of_Birth',
 'Age_at_sampling',
 'Ethnicity_cerner',
 'Diagnosis_date',
 'Diagnosis_year',
 'Years_from_diagnosis',
 'Age_at_diagnosis',
 'Sex',
 'Lupus_status',
 'IFN_score',
 'IFN_status',
 'SLEDAI_2K',
 'SLEDAI_2K_Comments',
 'SLEDAI_2KG',
 'SLEDAI_2K_Clinical',
 'cSLEDAI_2K',
 'LN',
 'LN_date',
 'LN_Class',
 'LN_Class_tidy',
 'Kidney_biopsies',
 'Organ_CVS',
 'Organ_Arterial',
 'Organ_Venous',
 'Organ_Lung_Fibrosis',
 'Multiple_Sclerosis',
 'Thyroid_Disease_Hashimotos',
 'Graves_Disease',
 'Type_I_Diabetes',
 'Rheumatoid_Arthritis',
 'Any_other_comorbidities',
 'MMF',
 'MMF_dose',
 'Pred',
 'Pred_dose',
 'HCQ',
 'HCQ_dose',
 'AZA',
 'AZA_dose',
 'MTX',
 'MTX_dose',
 'TAC',
 'TAC_dose',
 'ASP',
 'ASP_dose',
 'CYC_ever',
 'CYC_dose',
 'CYC_date',
 'CYC_cycles',
 'BEL_ever',
 'BEL_dose',
 'BEL_date',
 'RTX_ever',
 'RTX_dose',
 'RTX_date',
 'RTX_cycles',
 'OFA_ever',
 'OFA_dose',
 'OFA_date',
 'OFA_cycles',
 'dsDNA',
 'dsDNA_titre',
 'dsDNA_units',
 'dsDNA_Ab_ever',
 'dsDNA_date',
 'ANA',
 'ANA_ever',
 'Ribosomal_P',
 'Ro_52',
 'Ro_60',
 'La',
 'Centromere',
 'Sm',
 'Sm_RNP',
 'RNP_68',
 'RNP_A',
 'Scl70',
 'Jo_1',
 'MPO',
 'PR3',
 'LAC_status',
 'ACA_status',
 'ACA_IgG',
 'ACA_IgM',
 'B2G_status',
 'B2G_IgG',
 'B2G_IgM',
 'Vitamin_D',
 'Vitamin_D_date',
 'TSH',
 'TSH_date',
 'C3_level',
 'C4_level',
 'Comp_date',
 'uPCR',
 'uPCR_units',
 'uPCR_date',
 'Albumin',
 'Creatinine',
 'eGFR',
 'eGFR_adjust',
 'Haemoglobin',
 'Lymphopenia_now',
 'Lymphopenia_ever',
 'Lymphocyte_count',
 'Neutrophil_count',
 'Monocyte_count',
 'FBC_Biochem_date',
 'B_cell_n',
 'B_cell_%',
 'CD4_n',
 'CD4_%',
 'CD8_n',
 'CD8_%',
 'Lymph_subsets_date',
 'IgA',
 'IgG',
 'IgM',
 'Ig_date',
 'C3_status',
 'C4_status',
 'dsDNA_SLEDAI_status',
 'ACR_Malar',
 'ACR_Discoid',
 'ACR_Photosensitivity',
 'ACR_oral_ulcers',
 'ACR_Arthritis',
 'ACR_serositis',
 'ACR_seizures/psychosis',
 'ACR_Haem',
 'ACR_Immunologic',
 'KBX_2',
 'KBX_2_LN_Class',
 'KBX_3',
 'KBX_3_LN_Class',
 'KBX_4',
 'KBX_4_LN_Class',
 'KBX_5',
 'KBX_5_LN_Class',
 'KBX_6',
 'KBX_6_LN_Class',
 'KBX_7',
 'KBX_7_LN_Class']

In [34]:
both_sites = both_sites[new_order]

In [35]:
both_sites.to_csv(
    "/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/combined_data_tidied_30_10_24.csv", index = False)

# Check that calculated clinical SLEDAI matches that from Imperial

In [36]:
both_sites.query(
    "Recruit_centre == 'Imperial' and SLEDAI_2K_Clinical != cSLEDAI_2K")[["SLEDAI_2K","SLEDAI_2K_Clinical", "cSLEDAI_2K", "Patient_ID", "SLEDAI_2K_Comments"]]

,SLEDAI_2K,SLEDAI_2K_Clinical,cSLEDAI_2K,Patient_ID,SLEDAI_2K_Comments
110,NaN,NaN,NaN,L132,No_clinical_data
215,10.0,6.0,8.0,L259,Arthritis_Proteinuria
258,6.0,0.0,4.0,L69,Arthritis
315,NaN,NaN,NaN,SC7,NaN


In [37]:
both_sites.Ethnicity_cerner.value_counts()

Ethnicity_cerner
Asian.Indian                   33
White.Other                    28
Black.Caribbean                28
Black.African                  27
White.British                  25
African                        22
Other.anyotherethnicgroup      20
Black Caribbean                20
Asian.Other                    20
White-British                  18
White-other-European           11
Other.notstated                10
Black.Other                     5
Mixed.Other                     5
Asian.Chinese                   5
Indian                          4
White.Irish                     4
Mixed.White/BlackCaribbean      4
Mixed.White/Asian               4
Asian.Pakistani                 4
Mixed                           3
Asian.Bangladeshi               3
Any other Black background      2
Asian                           2
White & Black Caribbean         2
Mixed                           1
Any other Asian                 1
Any other Black background      1
Black-Carribean                

# Drop individuals not used for single cell sequencing

In [38]:
metadata = pd.read_csv("/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/metadata/SLEmap_metadata_191224.csv")

In [39]:
metadata.SC_included.value_counts()

SC_included
use     298
drop     13
Name: count, dtype: int64

In [40]:
metadata_sc_use = metadata.query("SC_included == 'use'").copy()

metadata_sc_use["externalID_stripr"] = metadata_sc_use["externalID"].str.replace("r","")

metadata_sc_use_samples = list(metadata_sc_use.externalID_stripr)

In [41]:
both_sites_sc_use = both_sites.query("externalID in @metadata_sc_use_samples")

In [42]:
both_sites_sc_use.to_csv(
    "/lustre/scratch126/opentargets/opentargets/OTAR2064/working/slemap/phenotype/combined_data_tidied_sc_used_25_11_24.csv", index = False)

# Healthy info

In [43]:
metadata_sc_use_healthy = metadata_sc_use.query("condition == 'HC'").copy()

In [44]:
metadata_sc_use_healthy

,externalID,experiment_id,SC_duplicate,SC_included,condition,recruitment_centre,recruitment_date,ancestry.l2,ancestry.l1,internalID,date_processed,post_thaw_count.10.6.,post_thaw_viability,pre.pool_count.10.6.,pre.pool_viability,pre_facs_count.10.6.,pre_facs_viability,facs_events.10.4.,post_facs_count.10.4.,pre_10X_viability,poolID,numberID,pool_cdna_conc_ngul,donor_cell_recovery,scSeq_run,scSeq_batch,cellranger_file,DNA_conc_totmicrogram,DNA_conc_QC,WGS_ID,WGS_EGAN_ID,WGS_batch,WGS_duplicate,vireo_assignment,externalID_stripr
15,EQTL066C,SLE_WGS13446469_LDP04,NaN,use,HC,Kings,NaN,White & Black African,Mixed ancestry,D1201,26/04/2022,6.99,68.2,4.35,55.2,2.55,58.7,281.0,210.0,83.0,LDP04,1201,3.2,2809.0,45271,1,cellranger700_multi_85b73be538c3c8c97a9f3cacf1...,NaN,passed,SLE_WGS13446469,EGAN00004257538,1.0,NaN,LDP04donor0,EQTL066C
23,EQTL054C,SLE_WGS13446470_LDP06,NaN,use,HC,Kings,NaN,White other European,European ancestry,D1202,27/04/2022,7.33,71.4,3.38,47.5,2.10,58.2,282.0,202.0,93.4,LDP06,1202,4.1,2339.0,45469,2,cellranger700_multi_45469_SLE_map12928660_and_...,NaN,passed,SLE_WGS13446470,EGAN00004257534,1.0,NaN,LDP06donor1,EQTL054C
37,EQTL035C,SLE_WGS13446471_LDP09,NaN,use,HC,Kings,NaN,White other European,European ancestry,D1203,09/05/2022,3.32,80.3,1.92,65.1,2.10,71.6,300.0,182.0,100.0,LDP09,1203,4.1,4173.0,45622,3,cellranger700_multi_a3cc8d52bb0f363c027d8c0783...,NaN,passed,SLE_WGS13446471,EGAN00004257535,1.0,NaN,LDP09donor1,EQTL035C
50,EQTL044C,SLE_donor4_LDP12,NaN,use,HC,Kings,NaN,White other European,European ancestry,D1205,10/05/2022,7.08,82.6,3.25,61.4,2.10,55.2,300.0,110.0,83.6,LDP12,1205,3.9,2499.0,45469,2,cellranger700_multi_45469_SLE_map12928666_and_...,2.09,passed,NaN,NaN,NaN,NaN,LDP12donor4,EQTL044C
74,EQTL063C,SLE_donor3_LDP17,NaN,use,HC,Kings,NaN,White other European,European ancestry,D1207,20/07/2022,3.41,78.6,2.59,78.8,2.00,64.0,350.0,244.0,86.3,LDP17,1207,7.9,3723.0,45916,4,cellranger700_multi_c9d76f90bbd437718946517c9e...,0.43,passed,NaN,NaN,NaN,NaN,LDP17donor3,EQTL063C
92,EQTL024C,SLE_WGS13446474_LDP21,NaN,use,HC,Kings,NaN,White British,European ancestry,D1208,25/07/2022,13.94,65.0,6.67,64.0,3.40,77.7,291.0,230.0,89.5,LDP21,1208,6.9,1795.0,45916,4,cellranger700_multi_0f2e6fd46960ceef6a6023494e...,NaN,passed,SLE_WGS13446474,EGAN00004257509,1.0,NaN,LDP21donor1,EQTL024C
129,EQTL049C,SLE_map13436235_LDP29,NaN,use,HC,Kings,NaN,Mixed,Mixed ancestry,D1211,07/11/2022,17.56,86.6,6.07,73.1,2.30,78.9,260.0,170.0,91.4,LDP29,1211,6.5,3245.0,46366,5,cellranger700_multi_bc45a1c2fe2a3fbbcde46cf984...,NaN,passed,SLE_map13436235,EGAN00004260909,2.0,NaN,LDP29donor4,EQTL049C
133,EQTL085C,SLE_donor3_LDP30,NaN,use,HC,Kings,NaN,White British,European ancestry,D1276,16/01/2023,9.27,88.0,5.20,60.8,2.40,60.4,313.0,222.0,91.2,LDP30,1276,6.2,NaN,46875,6,cellranger700_multi_b4f011068f70444b8985ca3169...,5.18,passed,NaN,NaN,NaN,NaN,LDP30donor3,EQTL085C
190,EQTL086C,SLE_donor4_LDP43,NaN,use,HC,Kings,NaN,Black Caribbean,Afro-Caribbean ancestry,D1272,31/01/2023,13.28,86.2,7.44,71.9,2.30,68.8,311.0,233.0,85.0,LDP43,1272,6.0,2033.0,46875,6,cellranger700_multi_e81952927d158ec7d499a13acf...,6.47,passed,NaN,NaN,NaN,NaN,LDP43donor4,EQTL086C
292,EQTL090C,SLE_donor3_LDP65,scDup3_use,use,HC,Kings,NaN,White European,European ancestry,D1278,19/04/2023,10.13,92.2,5.20,61.2,2.40,62.1,314.0,193.0,88.0,LDP65,1278,NaN,NaN,47425,9,cellranger700_multi_7310347ace80be8e9e40236d98...,NaN,NaN,NaN,NaN,NaN,NaN,LDP65donor3,EQTL090C


# King's samples with no C3/C4

In [45]:
c3_c4_dsdna = both_sites.copy()

In [46]:
c3_c4_dsdna_kings = c3_c4_dsdna.query("Recruit_centre == 'Kings'")

In [47]:
c3_c4_dsdna_kings = c3_c4_dsdna_kings[["Patient_ID", "Site", "Sample_date", "C3_level", "C4_level", "dsDNA"]]

In [48]:
c3_c4_dsdna_kings_missing = c3_c4_dsdna_kings[c3_c4_dsdna_kings.isnull().any(axis=1)]

In [49]:
c3_c4_dsdna_kings_missing = c3_c4_dsdna_kings_missing.fillna("No")

In [50]:
c3_c4_dsdna_kings_missing.to_excel("/nfs/users/nfs_c/cs54/OTAR2064_cs54/final_data/1_Metadata/Kings_samples_missing_C3_C4_dsDNA.xlsx")  

In [51]:
both_sites

,Patient_ID,externalID,Sample_ID,Site,Recruit_centre,Date_of_consent,Sample_date,Sample_year,Year_of_Birth,Age_at_sampling,Ethnicity_cerner,Diagnosis_date,Diagnosis_year,Years_from_diagnosis,Age_at_diagnosis,Sex,Lupus_status,IFN_score,IFN_status,SLEDAI_2K,SLEDAI_2K_Comments,SLEDAI_2KG,SLEDAI_2K_Clinical,cSLEDAI_2K,LN,LN_date,LN_Class,LN_Class_tidy,Kidney_biopsies,Organ_CVS,Organ_Arterial,Organ_Venous,Organ_Lung_Fibrosis,Multiple_Sclerosis,Thyroid_Disease_Hashimotos,Graves_Disease,Type_I_Diabetes,Rheumatoid_Arthritis,Any_other_comorbidities,MMF,MMF_dose,Pred,Pred_dose,HCQ,HCQ_dose,AZA,AZA_dose,MTX,MTX_dose,TAC,TAC_dose,ASP,ASP_dose,CYC_ever,CYC_dose,CYC_date,CYC_cycles,BEL_ever,BEL_dose,BEL_date,RTX_ever,RTX_dose,RTX_date,RTX_cycles,OFA_ever,OFA_dose,OFA_date,OFA_cycles,dsDNA,dsDNA_titre,dsDNA_units,dsDNA_Ab_ever,dsDNA_date,ANA,ANA_ever,Ribosomal_P,Ro_52,Ro_60,La,Centromere,Sm,Sm_RNP,RNP_68,RNP_A,Scl70,Jo_1,MPO,PR3,LAC_status,ACA_status,ACA_IgG,ACA_IgM,B2G_status,B2G_IgG,B2G_IgM,Vitamin_D,Vitamin_D_date,TSH,TSH_date,C3_level,C4_level,Comp_date,uPCR,uPCR_units,uPCR_date,Albumin,Creatinine,eGFR,eGFR_adjust,Haemoglobin,Lymphopenia_now,Lymphopenia_ever,Lymphocyte_count,Neutrophil_count,Monocyte_count,FBC_Biochem_date,B_cell_n,B_cell_%,CD4_n,CD4_%,CD8_n,CD8_%,Lymph_subsets_date,IgA,IgG,IgM,Ig_date,C3_status,C4_status,dsDNA_SLEDAI_status,ACR_Malar,ACR_Discoid,ACR_Photosensitivity,ACR_oral_ulcers,ACR_Arthritis,ACR_serositis,ACR_seizures/psychosis,ACR_Haem,ACR_Immunologic,KBX_2,KBX_2_LN_Class,KBX_3,KBX_3_LN_Class,KBX_4,KBX_4_LN_Class,KBX_5,KBX_5_LN_Class,KBX_6,KBX_6_LN_Class,KBX_7,KBX_7_LN_Class
0,EQTL009P,EQTL009P,NaN,Guys,Kings,27/10/2020,27/10/2020,2020,1978,42,African,2003,2003.0,17.0,25.0,NaN,NaN,NaN,NaN,4.0,Arthritis,NaN,NaN,4.0,Yes,2007,Class III,III,1,No,No,No,No,No,No,No,No,No,No,Yes,1 g daily,Yes,7.5mg OD,Yes,200mg daily,No,NaN,No,NaN,No,NaN,No,NaN,Yes,NaN,NaN,4,No,NaN,NaN,No,NaN,NaN,NaN,No,NaN,NaN,NaN,Yes,NaN,NaN,NaN,NaN,Positive,Positive,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,NaN,NaN,NaN,NaN,0.96,1.14,27/10/2020,NaN,NaN,NaN,40.0,83.0,65.0,65.0,131,No,Yes,1.2,3,0.5,27/10/2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,EQTL041P,EQTL041P,NaN,Guys,Kings,18/05/2021,18/05/2021,2021,1984,37,White-British,2014,2014.0,7.0,30.0,NaN,NaN,NaN,NaN,2.0,"Low C4,C3",NaN,NaN,0.0,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,NaN,No,NaN,Yes,400/200 AD,No,NaN,No,NaN,No,NaN,Yes,75mg daily,No,NaN,No,NaN,No,NaN,No,No,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,Positive,Negative,Positive,Positive,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Negative,66,18/05/2021,2.06,18/05/2021,0.39,0.04,18/05/2021,NaN,NaN,NaN,42.0,55.0,108.0,90.0,126,No,No,1.0,4,0.2,18/05/2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LOW,LOW,NORMAL,No,No,No,No,No,No,No,Yes,Yes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EQTL073P,EQTL073P,NaN,Guys,Kings,09/11/2021,09/11/2021,2021,1990,31,White-British,2014,2014.0,7.0,24.0,NaN,NaN,NaN,NaN,0.0,none,NaN,NaN,0.0,No,NaN,NaN,NaN,NaN,No,No,No,No,No,No,No,No,No,No,No,NaN,No,NaN,Yes,400mg daily,No,NaN,No,NaN,No,NaN,No,NaN,No,NaN,NaN,NaN,No,NaN,NaN,No,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Positive,Positive,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,Negative,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NORMAL,Yes,No,No,No,No,No,No,No,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,EQTL0119P,EQTL119P,NaN,Guys,Kings,15/11/2022,15/11/2022,2022,1988,34,Black-Carri

In [52]:
both_sites.Lupus_status.value_counts(dropna=False)

Lupus_status
INACTIVE    191
NaN          90
ACTIVE       35
Name: count, dtype: int64